# Introduction to Azure OpenAI Function Calling


## Objective

This notebook demonstrates the following:

1. Generative AI
1. Function calling


## Time

You should expect to spend 2 minutes running this sample.


## About this example

The objective of the provided Python file is to learn about Azure OpenAI Function Calling using the Azure OpenAI API.


## Installation

Refer to the README.md file in this folder for installation instructions.


## Parameters


In [14]:
import os
import json
import sqlite3
import pandas as pd
from openai import AzureOpenAI
from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Any, Callable, Dict

load_dotenv(".env")

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

con = sqlite3.connect("./database/order_data.db")

instructions = None

In [15]:
def get_revenue_by_region(region: str = None) -> pd.DataFrame:
    params = []
    query = """
        SELECT
            Region,
            SUM(Number_of_Orders) AS Number_of_Orders,
            CAST(SUM(Sum_of_Order_Value_USD) AS INTEGER) AS Total_Order_Value,
            SUM(Sum_of_Order_Value_USD) / SUM(Number_of_Orders) AS Avg_Order_Value,
            SUM(Sum_of_Shipping_Cost_USD) / SUM(Number_of_Orders) AS Avg_Shipping_Cost
        FROM order_data
    """

    if region:
        query += "WHERE UPPER(Region) = UPPER(?)\n"
        params.append(region)

    query += "GROUP BY Region ORDER BY Region"

    return pd.read_sql_query(query, con, params=params)

In [16]:
def get_sales_by_month(month: int) -> pd.DataFrame:
    query = """
        SELECT   
            SUM(Sum_of_Order_Value_USD) AS Revenue, Month, Year
            FROM order_data WHERE Month = ? GROUP BY Month, Year ORDER BY Year, Month
    """

    return pd.read_sql_query(query, con, params=[month])

### Create an AzureOpenAI client


In [17]:
client = AzureOpenAI(
    api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint
)

### Define the Assistant tools


In [18]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_revenue_by_region",
            "description": "Get the sales revenue for Contoso by region.",
            "parameters": {
                "type": "object",
                "properties": {
                    "region": {"type": "string"},
                },
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_sales_by_month",
            "description": "Get the Contoso sales by month.",
            "parameters": {
                "type": "object",
                "properties": {
                    "month": {
                        "type": "integer",
                        "description": "The Contoso sales for a specific month.",
                    }
                },
                "required": ["month"],
            },
        },
    },
]

### Process Function calling


In [19]:
def call_functions(tool_calls) -> None:
    function_map: Dict[str, Callable[[Any], pd.DataFrame]] = {
        "get_revenue_by_region": lambda args: get_revenue_by_region(args.get("region", None)),
        "get_sales_by_month": lambda args: get_sales_by_month(args["month"])
    }

    for tool_call in tool_calls:
        func_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        # Debugging output
        display(HTML(f'<span style="color: red;"><strong>Function Name: {func_name}, Function Args: {arguments}</strong></span>'))

        function = function_map.get(func_name)
        if not function:
            raise ValueError(f"Unknown function: {func_name}")
        
        result_df = function(arguments)
        display(result_df)

## Instructions for role system assignment


In [20]:
instructions = (
    "You are a sales analysis assistant for Contoso. "
    "Please be polite, professional, helpful, and friendly. "
    "You get all the sales data from this app using the functions provided. "
    "If a question is not related to sales or you cannot answer the question, "
    "say, 'contact IT for more assistance.' "
    "If the user asks for help or says 'help', provide a list of sample questions that you can answer."
)

## Construct the Assistant and send to Azure OpenAI


In [21]:
def process_message(question: str):

    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": question}
    ]

    response = client.chat.completions.create(
        model=api_deployment_name,
        messages=messages,
        tools=tools_list,
    )

    response_message = response.choices[0].message
    tool_calls = getattr(response_message, 'tool_calls', [])

    if tool_calls:
        call_functions(tool_calls)
    else:
        display(f"Response message: {response_message.content}")
        return

In [22]:
process_message("What is the revenue for the region of Africa?")

,Region,Number_of_Orders,Total_Order_Value,Avg_Order_Value,Avg_Shipping_Cost
0,AFRICA,61209,1533671,25.056306,7.507702


In [23]:
process_message("What is the revenue for the region of north america?")

,Region,Number_of_Orders,Total_Order_Value,Avg_Order_Value,Avg_Shipping_Cost
0,NORTH AMERICA,62555,1092029,17.457103,7.502805


In [24]:
process_message("What is the revenue for all regions?")

,Region,Number_of_Orders,Total_Order_Value,Avg_Order_Value,Avg_Shipping_Cost
0,AFRICA,61209,1533671,25.056306,7.507702
1,ASIA-PACIFIC,61337,1344150,21.914194,7.490754
2,EUROPE,61719,1199818,19.440016,7.502063
3,MIDDLE EAST,61168,1786876,29.212609,7.502444
4,NORTH AMERICA,62555,1092029,17.457103,7.502805
5,SOUTH AMERICA,61193,2147262,35.090006,7.475050


In [25]:
process_message("What were the sales for April?")

,Revenue,Month,Year
0,542127.134541,4,2023
1,370041.415045,4,2024


In [26]:
process_message("What is the meaning of life?")

"Response message: For philosophical or existential questions like this, it's best to explore personal and philosophical perspectives, or perhaps contact a philosophical or mental health professional. For assistance with sales data, feel free to ask me specific questions about Contoso's sales."